In [1]:
from src.llm_plan.environment import Environment

env = Environment("./environments/static/TripPlanning.json")
env.plan

[['traveler.pddl', 'friend.pddl'], ['orchestrator.pddl']]

In [2]:
# Create the plan
from src.llm_plan.planner import Planner
from src.llm_plan.llm import GPT_4o

model = GPT_4o()
planner = Planner(env, model)

In [3]:
# Plan
responses = planner.plan()

----- traveler->pddl -----

----- System Prompt -----
You are an expert with PDDL problems (Planning Domain Definition Language). You always provide a PDDL domain and a PDDL problem file to solve the task. You always enclose the pddl domain between <domain></domain> tags, and the pddl problem between <problem></problem> tags.

----- Prompt -----
You are the traveler planning a 14-day trip. Your goals are: Create a detailed 14-day trip itinerary visiting London, Bucharest, and Reykjavik with direct flights.. The constraints are: ['The trip spans 14 days.', 'The traveler plans to visit London, Bucharest, and Reykjavik.', 'Direct flights are available between London and Bucharest, and between London and Reykjavik.', 'The traveler intends to meet a friend in Bucharest between day 10 and day 14.']. Think step by step and provide a PDDL domain and a PDDL problem file to solve the task.
If you miss some information, do not make assumptions, just give a plan that concerns the information you h

In [4]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

# TODO: fix the vocabulary entry
final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [7]:
# Obtain the plan with the solver
import subprocess

from pathlib import Path

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

with open(BASE_FOLDER / "problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "sas_plan",
    BASE_FOLDER / "domain.pddl",
    BASE_FOLDER / "problem.pddl",
]

with open(BASE_FOLDER / "logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)

In [6]:
# Validate the plan with uVAL
import subprocess

from src.llm_plan.config import UNIVERSAL_VALIDATOR_BIN

command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'domain.pddl'} \
{BASE_FOLDER / 'problem.pddl'} \
{BASE_FOLDER / 'sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: LONDON does not name a known token

